# Secure Remote Password

We show here a naive implentation of the SRP protocol, a protocol that allows an user to login into a site without having the site know its password. The protocol consists of three phases, the preliminary phase, the registration phase and the logging phase. In the preliminary both parties agree to an Id $I$ for the user, a safe prime $N$ and a generator $g$ for the multiplicative group $(\mathbb{Z}_N)^\times$. For added security they also select a $k \in \mathbb(GF)(N)$.

Then, the registration phase works as follows:
1. The user selects a password $p$, and a random salt $s$
2. It computes $x \equiv H(s, p)$ and $v \equiv g^x$
3. It then sends $(s, v)$ to the host, which stores them with the identifier $I$

On authentication, the procedure is as follows:
1. The user selects a random $a$, computes $A \equiv g^a$ and sends $(I, A)$ to the host
2. The host lookups using $I$ the $(s, v)$ previosly stored, select a random $b$ and computes $B \equiv kv + g^b$. It then sends $(s, B)$ to the user
3. Both parties compute $u = H(A, B)$
4. The user computes $x \equiv H(s, p)$, and from it the session key $S \equiv (B - kg^x)^{a + ux}$. From it it can compute a key $K = H(S)$
5. The host computes $S \equiv (Av^u)^b$, and from it the key $K = H(S)$

If all steps are executed correctly, the $K$ that both parties receive will be equal. Furthermore, the security of the scheme is due to the fact that the discrete log is hard to invert in the group in question.  
First of all, we start by agreeing on our $N$, for which we use a safe prime i.e. a prime of the form $2p + 1$ where $p$ is prime. Then we select some generator and some random element $k$, plus a username that will be "Bob" for our purposes

In [71]:
bits = 128

def safe_prime(nbits):
    while True:
        p = random_prime(2^nbits-1, false, 2^(nbits-1))
        if ZZ((p+1)/2).is_prime():
            return p
        
N = safe_prime(bits)
F = FiniteField(N)
k = F.random_element()
g = F.multiplicative_generator()
I = 'Bob'

import hashlib
from sage.crypto.util import ascii_to_bin

def H(*args):
    hs = hashlib.sha1()
    for a in args:
        hs.update(str(a))
    return hs.hexdigest()

def hash_to_int(hs):
    return ZZ('0x' + hs)

Here is the registration step. We select a password of 128 bits, and some smaller salt. $(s, v)$ are the values that then the host receives, and we assume that he can access them at will

In [72]:
p = getrandbits(bits)
s = getrandbits(32)
x = hash_to_int(H(s, p))
v = g^x
(s, v)

(1598876508L, 53494898264320744629372320720535041848)

Now, we are in the auth phase. The user computes its random $A$ and sends it to host

In [73]:
a = getrandbits(bits)
A = g^a
(I, A)

('Bob', 194915951129395064026584438670225872096)

The host computes its $B$, lookups $(s, v)$ and sends it to the user

In [74]:
b = getrandbits(bits)
B = k * v + g^b
(s, B)

(1598876508L, 42498612804040704006477076953938202030)

Both parties now compute the hash $u$ (Note that since here we do not really have separation of variables we do it in one line for convinience)

In [75]:
u = hash_to_int(H(A, B))

Now the user computes its key using the method specified above

In [76]:
x = hash_to_int(H(s, p))
S_user = (B - k*g^x)^(a + u*x)
K_user = H(S)
K_user

'7db40c734db299fd9c564f63d18d0f9130622886'

The host does as well

In [77]:
S_host = (A * v^u)^b
K_host = H(S)
K_host

'7db40c734db299fd9c564f63d18d0f9130622886'

Finally, all that is left is to show that the keys are equal, that we can see below

In [78]:
K_user == K_host

True